In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import mynnlib
from mynnlib import *

dataset_dir = "insect-dataset/lepidoptera"

early_regex = r"^.*-(early)$"
unidentified_regex = r"^.*-(spp|genera|genera-spp)$"
early_or_unidentified_regex = r"^.*-(early|spp|genera|genera-spp)$"

# Create datasets

In [4]:
if os.path.exists(f"{dataset_dir}/data"):
    shutil.rmtree(f"{dataset_dir}/data")
os.makedirs(f"{dataset_dir}/data")
for src_dir in [f"{dataset_dir}/../moth/data", f"{dataset_dir}/../butterfly/data"]:
    for class_dir in os.listdir(src_dir):
        shutil.copytree(f"{src_dir}/{class_dir}", f"{dataset_dir}/data/{class_dir}")

In [5]:
if os.path.exists(f"{dataset_dir}/val"):
    shutil.rmtree(f"{dataset_dir}/val")
os.makedirs(f"{dataset_dir}/val")
for src_dir in [f"{dataset_dir}/../moth/val", f"{dataset_dir}/../butterfly/val"]:
    for class_dir in os.listdir(src_dir):
        shutil.copytree(f"{src_dir}/{class_dir}", f"{dataset_dir}/val/{class_dir}")

# Count

In [6]:
classes = { class_dir: len([ img for img in os.listdir(f"{dataset_dir}/data/{class_dir}") ]) for class_dir in os.listdir(f"{dataset_dir}/data") }
early_classes = { class_name: count for class_name, count in classes.items() if re.match(early_regex, class_name) }
unidentified_classes = { class_name: count for class_name, count in classes.items() if re.match(unidentified_regex, class_name) }
print(f"Total Class count : {len(classes):6} ( Unidentified: {len(unidentified_classes):6} / Early-stage: {len(early_classes):6} / Identified-adult: {len(classes) - len(unidentified_classes) - len(early_classes):6} )")
print(f"Total  Data count : {sum(classes.values()):6} ( Unidentified: {sum(unidentified_classes.values()):6} / Early-stage: {sum(early_classes.values()):6} / Identified-adult: {sum(classes.values()) - sum(unidentified_classes.values()) - sum(early_classes.values()):6} )")

Total Class count :   5177 ( Unidentified:    446 / Early-stage:    992 / Identified-adult:   3739 )
Total  Data count : 312320 ( Unidentified:  13020 / Early-stage:  25256 / Identified-adult: 274044 )


In [7]:
img2_class = []
img5_class = []
for class_dir in os.listdir(f"{dataset_dir}/data"):
    if not re.match(early_or_unidentified_regex, class_dir):
        img_cnt = sum([1 for file in os.listdir(f"{dataset_dir}/data/{class_dir}")])
        img2_class += [class_dir] if img_cnt <= 2 else []
        img5_class += [class_dir] if img_cnt <= 5 else []
print(f"{len(img2_class):6} classes with <=2 images")
print(f"{len(img5_class):6} classes with <=5 images")

   153 classes with <=2 images
   434 classes with <=5 images


In [8]:
generas = set()
for class_name in classes:
    generas.add(class_name.split('-')[0])
print(f"Genera count: {len(generas)}")

Genera count: 1813


# Combine class details files

In [8]:
import json

input_data = {
    "models/class_details.butterfly.json": {'singular': "Butterfly", 'plural': "Butterflies"},
    "models/class_details.moth.json": {'singular': "Moth", 'plural': "Moths"},
}
combined_data = {}

for src, details in input_data.items():
    with open(src, "r", encoding="utf-8") as file:
        data = json.load(file)
    for key, value in data.items():
        if not re.match(r"(?i)^.*" + details['singular'] + "|" + details['plural'] + r"$", value['name']):
            value['name'] += " " + details['singular']
        combined_data[key] = value

for species_type in ['moth', 'butterfly']:
    for class_name in os.listdir(f"{dataset_dir}/../{species_type}/data"):
        if not re.match(early_or_unidentified_regex, class_name) and class_name not in combined_data:
            name = re.sub(r"-", " ", class_name, count=1).title() + " " + species_type.title()
            combined_data[class_name] = { 'name' : name}

with open("models/class_details.lepidoptera.json", "w", encoding="utf-8") as file:
    json.dump(combined_data, file, indent=4)

# Train
### Model A (resnet-152)

In [16]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet152", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.lepidoptera.ta.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 2.1822 Acc: 0.5864  | Val Loss: 1.4265 Acc: 0.6265  | Elapsed time: 1:21:41.822510
Epoch    2 /    5  | Train Loss: 0.8145 Acc: 0.8010  | Val Loss: 1.2405 Acc: 0.7531  | Elapsed time: 2:40:29.224297
Epoch    3 /    5  | Train Loss: 0.6167 Acc: 0.8454  | Val Loss: 1.1244 Acc: 0.7346  | Elapsed time: 3:58:54.970532
Phase 2:
Epoch    1 /    5  | Train Loss: 1.5067 Acc: 0.6823  | Val Loss: 1.0881 Acc: 0.7562  | Elapsed time: 1:21:08.680264
Epoch    2 /    5  | Train Loss: 1.3111 Acc: 0.7191  | Val Loss: 1.0673 Acc: 0.7716  | Elapsed time: 2:41:32.846402
Phase 3:
Epoch    1 /    5  | Train Loss: 1.2929 Acc: 0.7225  | Val Loss: 1.1342 Acc: 0.7500  | Elapsed time: 1:20:42.444327
Epoch    2 /    5  | Train Loss: 1.2441 Acc: 0.7325  | Val Loss: 1.0588 Acc: 0.7747  | Elapsed time: 2:41:30.319693
Epoch    3 /    5  | Train Loss: 0.9636 Acc: 0.7913  | Val Loss: 0.8902 Acc: 0.8148  | Elapsed time: 4:02:22.746229
Epoch    4 /    5  | Train Loss: 0.8768 Acc: 

In [22]:
model_data = torch.load(f"{dataset_dir}/checkpoint.lepidoptera.ta.ep050004.pth", weights_only=False)

In [23]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 119/153 -> 77.78%, genus matched: 138/153 -> 90.20%
Top   3 accuracy: 135/153 -> 88.24%, genus matched: 150/153 -> 98.04%
Top   5 accuracy: 144/153 -> 94.12%, genus matched: 152/153 -> 99.35%
Top  10 accuracy: 145/153 -> 94.77%, genus matched: 153/153 -> 100.00%


In [24]:
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 130/152 -> 85.53%, genus matched: 146/152 -> 96.05%
Top   3 accuracy: 147/152 -> 96.71%, genus matched: 147/152 -> 96.71%
Top   5 accuracy: 147/152 -> 96.71%, genus matched: 147/152 -> 96.71%
Top  10 accuracy: 147/152 -> 96.71%, genus matched: 147/152 -> 96.71%


In [25]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

acraea-terpsicore             : acraea-terpsicore(0.971)  lycaena-panava(0.016)  argynnis-hybrida(0.008)  
athyma-pravara                : neptis-clinia(0.648)  neptis-nata(0.158)  neptis-soma(0.044)  
colias-fieldii                : colias-fieldii(0.998)  colias-eogene(0.001)  colias-erate(0.001)  
danaus-melanippus             : danaus-melanippus(0.926)  danaus-genutia(0.070)  danaus-chrysippus(0.000)  
delias-descombesi             : delias-descombesi(0.944)  delias-agostina(0.008)  delias-pasithoe(0.005)  
euploea-core                  : euploea-core(0.431)  euploea-sylvester(0.267)  euploea-algea(0.144)  
graphium-doson                : graphium-doson(0.982)  graphium-teredon(0.008)  graphium-sarpedon(0.005)  
hypolimnas-bolina             : hypolimnas-bolina(0.999)  hypolimnas-misippus(0.001)  mimathyma-ambica(0.000)  
kallima-inachus               : kallima-inachus(0.997)  doleschallia-bisaltide(0.001)  kallima-albofasciata(0.000)  
lethe-verma                   : lethe-verma(0.

In [20]:
test_top_k(model_data, f"{dataset_dir}/../moth/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

apona-spp                     : apona-caschmirensis(0.382)  polyptychus-trilineatus(0.232)  marumba-dyras(0.101)  
dysphania-percota             : dysphania-percota(0.997)  dysphania-percota-early(0.000)  teliphasa-albifusa(0.000)  
eupterote-undata              : eupterote-undata(0.990)  eupterote-spp(0.005)  eupterote-mollifera(0.002)  
hippotion-rosetta-2           : hippotion-boerhaviae(0.536)  hippotion-rosetta(0.226)  cechetra-minor(0.154)  
hippotion-rosetta             : hippotion-rosetta(0.396)  hippotion-boerhaviae(0.364)  hippotion-spp(0.176)  
----------
Top   1 accuracy: 3/5 -> 60.00%, genus matched: 5/5 -> 100.00%
Top   3 accuracy: 4/5 -> 80.00%, genus matched: 5/5 -> 100.00%


### Model B (resnet-101 + sorted data)
inaturalist data had lots of early stage data mixed in imago class, have been sorted now manually

In [9]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet101", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.lepidoptera.tb.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 2.2218 Acc: 0.5835  | Val Loss: 1.3690 Acc: 0.7037  | Elapsed time: 1:06:34.353595
Epoch    2 /    5  | Train Loss: 0.8291 Acc: 0.8009  | Val Loss: 1.1166 Acc: 0.7407  | Elapsed time: 2:11:51.312474
Phase 2:
Epoch    1 /    5  | Train Loss: 1.5705 Acc: 0.6715  | Val Loss: 1.0159 Acc: 0.7469  | Elapsed time: 1:07:39.699875
Epoch    2 /    5  | Train Loss: 1.3733 Acc: 0.7075  | Val Loss: 0.9844 Acc: 0.7593  | Elapsed time: 2:15:28.483240
Phase 3:
Epoch    1 /    5  | Train Loss: 1.3445 Acc: 0.7138  | Val Loss: 0.9940 Acc: 0.7716  | Elapsed time: 1:07:17.710645
Epoch    2 /    5  | Train Loss: 1.2949 Acc: 0.7240  | Val Loss: 1.0233 Acc: 0.7531  | Elapsed time: 2:14:56.035088
Phase 4:
Epoch    1 /    5 

KeyboardInterrupt: 

### Model C (resnet-152 + sorted data)

In [10]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet152", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.lepidoptera.tc.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 2.1977 Acc: 0.5879  | Val Loss: 1.2671 Acc: 0.6944  | Elapsed time: 1:20:03.938434
Epoch    2 /    5  | Train Loss: 0.8228 Acc: 0.8023  | Val Loss: 1.1125 Acc: 0.7284  | Elapsed time: 2:41:38.858001
Phase 2:
Epoch    1 /    5  | Train Loss: 1.5379 Acc: 0.6765  | Val Loss: 1.0099 Acc: 0.7654  | Elapsed time: 1:23:44.811123
Epoch    2 /    5  | Train Loss: 1.3459 Acc: 0.7117  | Val Loss: 1.0590 Acc: 0.7469  | Elapsed time: 2:47:16.850669
Phase 3:
Epoch    1 /    5  | Train Loss: 1.3132 Acc: 0.7188  | Val Loss: 1.0038 Acc: 0.7685  | Elapsed time: 1:24:05.802611
Epoch    2 /    5  | Train Loss: 1.2663 Acc: 0.7291  | Val Loss: 1.0119 Acc: 0.7346  | Elapsed time: 2:48:06.307405
Phase 4:
Epoch    1 /    5  | Train Loss: 1.3079 Acc: 0.7206  | Val Loss: 1.0882 Acc: 0.7469  | Elapsed time: 2:25:26.729687
Epoch    2 /    5  | Train Loss: 1.0291 Acc: 0.7792  | Val Loss: 0.8902 Acc: 0.7901  | Elapsed time: 3:46:25.230717
Epoch    3 /    5  | Train Loss: 0.9

In [11]:
model_data = torch.load(f"{dataset_dir}/checkpoint.lepidoptera.tc.ep040001.pth", weights_only=False)

In [12]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 112/153 -> 73.20%, genus matched: 133/153 -> 86.93%
Top   3 accuracy: 138/153 -> 90.20%, genus matched: 149/153 -> 97.39%
Top   5 accuracy: 142/153 -> 92.81%, genus matched: 152/153 -> 99.35%
Top  10 accuracy: 147/153 -> 96.08%, genus matched: 153/153 -> 100.00%


reduced top 1 accuracy, but increased top 3 accuracy

In [13]:
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 129/152 -> 84.87%, genus matched: 146/152 -> 96.05%
Top   3 accuracy: 145/152 -> 95.39%, genus matched: 147/152 -> 96.71%
Top   5 accuracy: 147/152 -> 96.71%, genus matched: 147/152 -> 96.71%
Top  10 accuracy: 148/152 -> 97.37%, genus matched: 148/152 -> 97.37%


In [14]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

acraea-terpsicore             : acraea-terpsicore(0.998)  danaus-chrysippus(0.001)  lycaena-panava(0.000)  
athyma-pravara                : neptis-nata(0.347)  neptis-clinia(0.297)  neptis-harita(0.109)  
colias-fieldii                : colias-fieldii(0.989)  colias-erate(0.008)  colias-nilagiriensis(0.001)  
danaus-melanippus             : danaus-melanippus(0.730)  danaus-genutia(0.265)  danaus-chrysippus(0.001)  
delias-descombesi             : delias-descombesi(0.962)  delias-pasithoe(0.006)  delias-eucharis(0.005)  
euploea-core                  : euploea-sylvester(0.258)  euploea-core(0.208)  euploea-godartii(0.198)  
graphium-doson                : graphium-doson(0.964)  graphium-sarpedon(0.012)  graphium-teredon(0.010)  
hypolimnas-bolina             : hypolimnas-bolina(0.994)  hypolimnas-misippus(0.003)  mimathyma-ambica(0.000)  
kallima-inachus               : kallima-inachus(1.000)  kallima-albofasciata(0.000)  doleschallia-bisaltide(0.000)  
lethe-verma                   : l

In [15]:
test_top_k(model_data, f"{dataset_dir}/../moth/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

apona-spp                     : apona-caschmirensis(0.416)  marumba-dyras(0.260)  polyptychus-trilineatus(0.127)  
dysphania-percota             : dysphania-percota(0.990)  dysphania-percota-early(0.002)  pachyodes-haemataria(0.002)  
eupterote-undata              : eupterote-undata(0.993)  eupterote-spp(0.005)  eupterote-mollifera(0.001)  
hippotion-rosetta-2           : hippotion-boerhaviae(0.363)  hippotion-rosetta(0.284)  cechetra-minor(0.219)  
hippotion-rosetta             : hippotion-boerhaviae(0.511)  hippotion-rosetta(0.279)  theretra-boisduvalii(0.051)  
----------
Top   1 accuracy: 2/5 -> 40.00%, genus matched: 5/5 -> 100.00%
Top   3 accuracy: 4/5 -> 80.00%, genus matched: 5/5 -> 100.00%
